# CMORising ACCESS-ESM1.6 Spin-Up for ACCESS-ENSO-Recipes

This notebook is a copy from a tutorial in [ACCESS-MOPPeR](https://github.com/ACCESS-NRI/ACCESS-MOPPeR)

The ACCESS CMIP7 consortium is actively developing the ACCESS-ESM1.6 model for submission to the CMIP7 Fast track initiative. As part of this effort, a spin-up run is being conducted on the NCI Gadi supercomputer, with output data progressively becoming available. However, this data cannot yet be easily evaluated using established evaluation frameworks such as [ESMValTool](https://www.esmvaltool.org/) or [ILAMB](https://www.ilamb.org/).

This notebook demonstrates how to process the variables required by the [ACCESS-ENSO-Recipe](https://github.com/ACCESS-NRI/ACCESS-ENSO-recipes), developed by ACCESS-NRI and leveraging ESMValTool, to enable robust evaluation of ACCESS-ESM1.6 spin-up runs. This workflow provides a pathway for the scientific assessment and validation of the new model output.

## Variables Required for ACCESS-NRI ENSO Recipes

To run the [ACCESS-NRI ENSO Recipes](https://github.com/ACCESS-NRI/ACCESS-ENSO-recipes) on the current ACCESS-ESM1.6 spin-up (CMIP7 FastTrack), the following variables are needed:

- **pr**: Precipitation
- **tos**: Sea surface temperature (can be interchanged with `ts`)
- **tauu**: Zonal wind stress
- **ts**: Surface air temperature (can be interchanged with `tos`)
- **zos**: Sea surface height
- **nhf**: Net heat flux, calculated as  
    `nhf = (-hfls - hfss + rlds - rlus + rsds - rsus)`

**Notes:**

- `ts` and `tos` can be used interchangeably.

This list is based on the requirements for the ENSO diagnostics and workflow as described in the ACCESS-NRI ENSO Recipes documentation.

To run the CMORIsation we advise using an X-Large or XX-Large session on ARE.

**Note:**  
ARE (Analysis Research Environment) is the interactive computing system provided by NCI (National Computational Infrastructure). Using a larger session (X-Large or XX-Large) ensures sufficient memory and CPU resources for efficient processing of large climate datasets.

In [1]:
from access_mopper import ACCESS_ESM_CMORiser


Loaded Configuration:
Creator Name: felicity Chun
Organisation: ACCESS-NRI
Creator Email: felicity.chun@anu.edu.au
Creator URL: 0009-0007-0845-0953


In [2]:
import dask.distributed as dask

client = dask.Client(threads_per_worker = 1)
client

/g/data/xp65/public/apps/med_conda/envs/analysis3-25.09/lib/python3.11/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 42455 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/42455/status,
Dashboard: /proxy/42455/status,Workers: 7
Total threads: 7,Total memory: 32.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:34519,Workers: 0
Dashboard: /proxy/42455/status,Total threads: 0
Started: Just now,Total memory: 0 B
Comm: tcp://127.0.0.1:38627,Total threads: 1
Dashboard: /proxy/43267/status,Memory: 4.57 GiB
Nanny: tcp://127.0.0.1:44611,


We are currently using the June Spin Up dataset located at:

`/g/data/p73/archive/CMIP7/ACCESS-ESM1-6/spinup/JuneSpinUp-JuneSpinUp-bfaa9c5b/`

This folder contains the output files from the ACCESS-ESM1.6 spin-up experiment, which are used throughout this notebook for CMORisation and evaluation.

In [3]:
ROOT_FOLDER = "/g/data/p73/archive/CMIP7/ACCESS-ESM1-6/spinup/JuneSpinUp-JuneSpinUp-bfaa9c5b/"

In [4]:
OUTPUT_FOLDER = "/scratch/nf33/fc6164/MOPPeR"

In [5]:
import glob
FILES = glob.glob(ROOT_FOLDER + "output[0-9][0-9][0-9]/atmosphere/netCDF/*mon.nc")

In [6]:
len(FILES), FILES[:5]


(12000,
 ['/g/data/p73/archive/CMIP7/ACCESS-ESM1-6/spinup/JuneSpinUp-JuneSpinUp-bfaa9c5b/output877/atmosphere/netCDF/aiihca.pa-196911_mon.nc',
  '/g/data/p73/archive/CMIP7/ACCESS-ESM1-6/spinup/JuneSpinUp-JuneSpinUp-bfaa9c5b/output877/atmosphere/netCDF/aiihca.pa-196906_mon.nc',
  '/g/data/p73/archive/CMIP7/ACCESS-ESM1-6/spinup/JuneSpinUp-JuneSpinUp-bfaa9c5b/output877/atmosphere/netCDF/aiihca.pa-196909_mon.nc',
  '/g/data/p73/archive/CMIP7/ACCESS-ESM1-6/spinup/JuneSpinUp-JuneSpinUp-bfaa9c5b/output877/atmosphere/netCDF/aiihca.pa-196901_mon.nc',
  '/g/data/p73/archive/CMIP7/ACCESS-ESM1-6/spinup/JuneSpinUp-JuneSpinUp-bfaa9c5b/output877/atmosphere/netCDF/aiihca.pa-196904_mon.nc'])

In [7]:
parent_experiment_config = {
        "parent_experiment_id": "piControl",
        "parent_activity_id": "CMIP",
        "parent_source_id": "ACCESS-ESM1-5",
        "parent_variant_label": "r1i1p1f1",
        "parent_time_units": "days since 0001-01-01 00:00:00",
        "parent_mip_era": "CMIP6",
        "branch_time_in_child": 0.0,
        "branch_time_in_parent": 54786.0,
        "branch_method": "standard"
}

### Amon.pr

In [9]:
# use output folder number to restrict the number of years to cmorise at a time ie output[0-2][0-9][0-9]

FILES = glob.glob(ROOT_FOLDER + "output[0-1][0-9][0-9]/atmosphere/netCDF/*mon.nc")
len(FILES)

2400

In [10]:
cmoriser = ACCESS_ESM_CMORiser(
    input_paths=FILES,
    compound_name="Amon.pr",
    experiment_id="piControl",
    source_id="ACCESS-ESM1-5",
    variant_label="r1i1p1f1",
    grid_label="gn",
    activity_id="CMIP",
    output_path=OUTPUT_FOLDER, # to write out file to nc
    parent_info=parent_experiment_config # <-- This is optional, can be skipped if not needed
    )

In [11]:
cmoriser.run()

# can get a cube directly from cmoriser (to use with ESMValCore preprocessors)
cube = cmoriser.to_iris()[0] #to_iris() gives a CubeList of one cube 

/g/data/xp65/public/apps/med_conda/envs/analysis3-25.09/lib/python3.11/site-packages/iris/fileformats/cf.py:664: IrisCfMissingVarWarning: Missing CF-netCDF grid mapping variable 'latitude_longitude', referenced by netCDF variable 'pr'
  warnings.warn(
/g/data/xp65/public/apps/med_conda/envs/analysis3-25.09/lib/python3.11/site-packages/iris/fileformats/cf.py:880: IrisCfMissingVarWarning: Missing CF-netCDF measure variable 'areacella', referenced by netCDF variable 'pr'
  warnings.warn(


In [ ]:
# can also get an xarray dataset
ds = cmoriser.to_dataset()

In [ ]:
cmoriser.write() # write file out to output folder

CMORised output written to /scratch/p66/fc6164/MOPPeR/pr_Amon_ACCESS-ESM1-5_piControl_r1i1p1f1_gn_159201-209112.nc


### Omon.tos

In [ ]:
FILES = glob.glob(ROOT_FOLDER + "output[0-2][0-9][0-9]/ocean/ocean-2d-surface_temp-1monthly-mean*.nc")
len(FILES)

500

In [20]:
cmoriser = ACCESS_ESM_CMORiser(
    input_paths=FILES,
    compound_name="Omon.tos",
    experiment_id="piControl",
    source_id="ACCESS-ESM1-5",
    variant_label="r1i1p1f1",
    grid_label="gn",
    activity_id="CMIP",
    parent_info=parent_experiment_config, # <-- This is optional, can be skipped if not needed
    output_path=OUTPUT_FOLDER
    )

In [21]:
cmoriser.run()

In [22]:
cmoriser.write()

CMORised output written to /scratch/p66/fc6164/MOPPeR/tos_Omon_ACCESS-ESM1-5_piControl_r1i1p1f1_gn_159201-209112.nc


### Amon.tauu

In [ ]:
FILES = glob.glob(ROOT_FOLDER + "output[0-2][0-9][0-9]/atmosphere/netCDF/*mon.nc")
len(FILES)

6000

In [24]:
cmoriser = ACCESS_ESM_CMORiser(
    input_paths=FILES,
    compound_name="Amon.tauu",
    experiment_id="piControl",
    source_id="ACCESS-ESM1-5",
    variant_label="r1i1p1f1",
    grid_label="gn",
    activity_id="CMIP",
    output_path=OUTPUT_FOLDER,
    parent_info=parent_experiment_config # <-- This is optional, can be skipped if not needed
    )

In [25]:
cmoriser.run()

In [26]:
cmoriser.write()

CMORised output written to /scratch/p66/fc6164/MOPPeR/tauu_Amon_ACCESS-ESM1-5_piControl_r1i1p1f1_gn_159201-209112.nc


### Amon.ts

In [ ]:
FILES = glob.glob(ROOT_FOLDER + "output[0-2][0-9][0-9]/atmosphere/netCDF/*mon.nc")
len(FILES)

6000

In [28]:
cmoriser = ACCESS_ESM_CMORiser(
    input_paths=FILES,
    compound_name="Amon.ts",
    experiment_id="piControl",
    source_id="ACCESS-ESM1-5", #controlled variable
    variant_label="r1i1p1f1",
    grid_label="gn",
    activity_id="CMIP",
    output_path=OUTPUT_FOLDER,
    parent_info=parent_experiment_config # <-- This is optional, can be skipped if not needed
    )

In [29]:
cmoriser.run()

In [30]:
cmoriser.write()

CMORised output written to /scratch/p66/fc6164/MOPPeR/ts_Amon_ACCESS-ESM1-5_piControl_r1i1p1f1_gn_159201-209112.nc


### Omon.zos

In [ ]:
FILES = glob.glob(ROOT_FOLDER + "output[0-2][0-9][0-9]/ocean/ocean-2d-sea_level-1monthly-mean*.nc")
len(FILES)

500

In [32]:
cmoriser = ACCESS_ESM_CMORiser(
    input_paths=FILES,
    compound_name="Omon.zos",
    experiment_id="piControl",
    source_id="ACCESS-ESM1-5",
    variant_label="r1i1p1f1",
    grid_label="gn",
    activity_id="CMIP",
    output_path=OUTPUT_FOLDER,
    parent_info=parent_experiment_config # <-- This is optional, can be skipped if not needed
    )

In [33]:
cmoriser.run()

In [ ]:
cmoriser.write()